<a href="https://colab.research.google.com/github/y2236li/Kiwi-Reasearch/blob/master/Test_2D_Climber_on_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.preprocessing import MinMaxScaler
import os
import tensorflow as tf
import collections
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense, Activation
from keras.models import Sequential, load_model
from keras.optimizers import Adam, SGD

Using TensorFlow backend.


In [0]:
%load_ext autoreload

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd drive/My\ Drive/kiwi_research/kiwi_project

/content/drive/My Drive/kiwi_research/kiwi_project


In [0]:
f_xy = open("pickles/XYtraintest.pk1", "rb")
X_train, X_test, Y_train, Y_test = pickle.load(f_xy)
f_xy.close()

## Agent

In [0]:
class agent():
  def __init__(self, param_name, detect_range, step_options, args):
    ## update at initialization
    self.param_name = param_name
    self.detect_range = detect_range
    self.step_options = step_options

    ## update every step
    self.cur_val = args[param_name]
    self.args = args
    self.cur_gre = np.zeros(len(detect_range))
    self.cur_dct = np.zeros(len(detect_range))
    self.cur_state = np.zeros(len(detect_range)*2+1)

    #memory
    self.state_mem = {}
    self.onehot_action_mem = {}
    self.reward_nor_mem = {}
    self.next_state_mem = {}
    self.terminal_mem = {}
    self.visited_val = {}

    ##scalers
    self.state_scaler = MinMaxScaler()
    self.reward_scaler = MinMaxScaler()


  def updateArg(self, cur_val, cur_gre, cur_dct):
    self.cur_val = cur_val
    self.cur_gre = cur_gre
    self.cur_dct = cur_dct
    self.args[self.param_name] = cur_val
    self.cur_state = np.array(list(cur_dct) + list(cur_gre) + list(self.args.values()))

  def updateMdl(self, q_model):
    self.q_model = q_model

  def resetPos(self, args, XY):
    global param_range

    self.cur_val = args[param_name]
    self.args = args
    self.cur_gre = np.zeros(len(detect_range))
    self.cur_dct = detect(self.param_name, self.detect_range, param_range[self.param_name], args, XY)
    self.updateArg(a.cur_val, a.cur_gre, a.cur_dct)



  def __enter__(self):
    return self
  
  def __exit__(self, type, value, traceback):
    pass

pickle_in = open("pickles/trained_agents.pk1","rb")
agents = pickle.load(pickle_in)

#Functions

In [0]:
def calAccuracy(args, XY):
  """
  Calculate the accuracy
  return: accuracy
  """
  global accuracy_lookuptb
  global mdl


  assert str(args) not in accuracy_lookuptb

  X_train, X_test, Y_train, Y_test = XY
  m = mdl(**args)
  m.fit(X_train, Y_train)
  Y_pred = m.predict(X_test)
  y = r2_score(Y_pred, Y_test)
  accuracy_lookuptb[str(args)] = y

  return y


def calReward(args, mdl, XY = None):
  global accuracy_lookuptb

  args_str = str(args)
  if args_str not in accuracy_lookuptb:
    assert XY
    return calAccuracy(args, XY)
  return accuracy_lookuptb[args_str]

def detect(param_name, detect_range, param_range,  args, XY = None):
  """
  return the accuracy of detecting area around the x
  
  """
  global accuracy_lookuptb
  global mdl

  assert param_name in args
  
  param_detect_range = np.array(detect_range) + args[param_name]
  param_detect_range = [max(min(x, max(param_range)), min(param_range)) for x in param_detect_range]

  args_list = []
  for pos in param_detect_range:
    args_list.append(args.copy())
    args_list[-1][param_name] = pos


  res = []
  for arg in args_list:
    if str(arg) in accuracy_lookuptb:
      res.append(accuracy_lookuptb[str(arg)])
    else:
      assert XY
      res.append(calAccuracy(arg, XY))
  
  return res



def step(a, action, param_range, XY = None):
  global mdl
  global last_mom
  global mom_ind
  global accuracy_lookuptb

  cur_val = a.cur_val
  args = a.args
  param_name = a.param_name
  detect_range = a.detect_range
  cur_detection = a.cur_dct
  last_mom = a.cur_gre


  cur_x = args[param_name]

  next_x = max(min(max(param_range), cur_x + action), min(param_range))

  args[param_name] = next_x

  if str(args) not in accuracy_lookuptb:
    assert XY
    next_detection = detect(param_name, detect_range, param_range, args, XY)
  else:
    next_detection = detect(param_name, detect_range, param_range, args, XY)
  
  next_gre = np.array(next_detection) - np.array(cur_detection) + np.array(last_mom) * mom_ind

  next_state = np.array(list(next_detection) + list(next_gre) + list(args.values()))

  reward = calReward(args, mdl, XY)

  a.updateArg(next_x, next_gre, next_detection)

  return (next_state, next_x, reward, a)




def writeMem(cur_state, next_state, action, reward, a):
  global mem_ctr
  index = mem_ctr % mem_size

  if isinstance(a.state_mem, dict) or (a.state_mem.shape[0] > index):
    a.state_mem[index] = cur_state
    a.next_state_mem[index] = next_state
    tmp_onehot = np.zeros(len(a.step_options), dtype=np.int64)
    tmp_onehot[list(a.step_options).index(action)] = 1
    a.onehot_action_mem[index] = tmp_onehot
    a.reward_nor_mem[index] = reward
    return a
    
  a.state_mem = np.append(a.state_mem, [cur_state], axis=0)
  a.next_state_mem = np.append(a.next_state_mem, [next_state], axis=0)
  tmp_onehot = np.zeros(len(a.step_options), dtype=np.int64)
  tmp_onehot[list(a.step_options).index(action)] = 1
  a.onehot_action_mem = np.append(a.onehot_action_mem, [tmp_onehot], axis=0)
  a.reward_nor_mem = np.append(a.reward_nor_mem, reward)
  return a


def norState(cur_state_dtc, cur_state_gre, cur_state_x, scaler_state):
  scaler_state_dtc, scaler_state_gre, scaler_state_x = scaler_state

  cur_state = np.array(list(cur_state_dtc) + list(cur_state_gre) + cur_state_x)

  cur_nor_state_dtc = scaler_state_dtc.transform([cur_state_dtc])[0]
  cur_nor_state_gre = scaler_state_gre.transform([cur_state_gre])[0]
  cur_nor_state_x = scaler_state_x.transform([cur_state_x])[0]

  cur_nor_state = np.array(list(cur_nor_state_dtc) + list(cur_nor_state_gre) + list(cur_nor_state_x))

  return cur_state, cur_nor_state



def calAccuracyTest(x):
  """
  Calculate the accuracy
  return: accuracy
  """
  pls2 = PLSRegression(n_components=x)
  pls2.fit(X_train_test, Y_train_test)
  Y_pred = pls2.predict(X_test_test)
  y = r2_score(Y_pred, Y_test_test)
  return y

# def calRewardTest(x):
#   return accuracy_lookuptb_test[x]


# def detectTest(detect_range, x, accuracy_lookuptb_test = None):
#   """
#   return the accuracy of detecting area around the x
#   """
#   x_detect_range = np.array(detect_range, dtype=np.int64) + int(x)
#   if not accuracy_lookuptb_test:
#     return [calAccuracyTest(pos) if pos in x_range else 0 for pos in x_detect_range]
#   return [accuracy_lookuptb_test[pos] if pos in x_range else 0 for pos in x_detect_range]



# def stepTest(cur_state, action, cur_x, accuracy_lookuptb_test):
#   next_x = int(max(min(max(x_range), cur_x + action), 1))
#   next_state = detectTest(detect_range, next_x, accuracy_lookuptb_test)
#   reward = calRewardTest(next_x)
#   return (next_state, reward)

## Initialization

In [0]:
mdl = SVR
gamma = 0.8
mom_ind = 0.8

tuning_args = ["C", "gamma"]

param_range = {}
unit = 100
param_range["C"] = range(100, 10000, unit)
unit = 1
param_range["gamma"] = np.arange(1, 1/(np.var(X_train)*X_train.shape[1]), unit)

## Start Testing

In [56]:
## Generate a new map
X = np.concatenate((X_train, X_test))
Y = np.concatenate((Y_train, Y_test))
np.random.seed(0)
train_indices = np.random.choice(X.shape[0], 300, replace=False)
test_indices = np.array([x for x in range(X.shape[0]) if x not in train_indices])
X_train = X[train_indices]
X_test = X[test_indices]
Y_train = Y[train_indices]
Y_test = Y[test_indices]
XY = (X_train, X_test, Y_train, Y_test)

#Lookup table for accuracy
accuracy_lookuptb = {}

#initial
init_args = {'C': 150, 'gamma': 100}
master_args = init_args.copy()
testing_his_max_reward = {}
n_mstr_round = 10
plot_path = False

#reset agents' position
for k in tuning_args:
  with agents[k] as a:
    a.cur_val = master_args.copy()[k]
    a.cur_dct = detect(a.param_name, a.detect_range, param_range[k], master_args, XY)
    a.cur_gre = np.zeros(len(a.detect_range))
    a.updateArg(a.cur_val, a.cur_gre, a.cur_dct)



for n_mstr_ctr in range(n_mstr_round): #change master_args

  max_args = {}

  for param_name in tuning_args: #for each agent
    a = agents[param_name]
    eplr_period = len(param_range[param_name])//2

    tmp_max_args = master_args.copy()
    tmp_max_args[a.param_name] += a.step_options[np.argmax(a.cur_dct)]
    tmp_max_reward = calReward(tmp_max_args.copy(), mdl, XY)

    for step_ctr in tqdm(range(eplr_period)):
      cur_nor_state = a.state_scaler.transform([a.cur_state])[0]
      action = a.step_options[np.argmax(a.q_model.predict(cur_nor_state.reshape(1, len(a.cur_state))))]
      next_state, next_val, reward, a = step(a, action, param_range[a.param_name], XY)
      if plot_path:
        plt.scatter(next_val, reward, c='r', alpha=max(0.2, 1/(step_ctr+1)))

      ## check if is max
      if reward > tmp_max_reward:
        tmp_max_reward = reward
        tmp_max_args = a.args.copy()

    ## when all game finished, reward the max_reward and max args
    max_args[param_name] = tmp_max_args
    testing_his_max_reward[str(tmp_max_args)] = (tmp_max_reward)

    ## plot path
    if plot_path:
      actual_acc_list = []
      for c in tqdm(param_range[param_name]):
        tmp_args = a.args.copy()
        tmp_args[param_name] = c
        actual_acc_list.append(calReward(tmp_args, mdl, XY))
      plt.plot(param_range[param_name], actual_acc_list)
      plt.show()

  for param_name in tuning_args:
    master_args[param_name] = max_args[param_name][param_name]
  print("Cur Master = ", master_args, " With reward = ", calReward(master_args, mdl, XY))



Cur Master =  {'C': 7850, 'gamma': 104.0}  With reward =  0.8926697480581488



Cur Master =  {'C': 6750, 'gamma': 93.0}  With reward =  0.892332588445216



Cur Master =  {'C': 5650, 'gamma': 82.0}  With reward =  0.8937460605532767



Cur Master =  {'C': 4550, 'gamma': 71.0}  With reward =  0.8933940817459128



Cur Master =  {'C': 9900, 'gamma': 104.0}  With reward =  0.8908904046136144



Cur Master =  {'C': 8800, 'gamma': 93.0}  With reward =  0.8933939717955901



Cur Master =  {'C': 7700, 'gamma': 82.0}  With reward =  0.8942763139491539



Cur Master =  {'C': 6600, 'gamma': 71.0}  With reward =  0.895703656067921



Cur Master =  {'C': 5500, 'gamma': 60.0}  With reward =  0.893639770495903



Cur Master =  {'C': 9900, 'gamma': 104.0}  With reward =  0.8908904046136144


## Calculate Exploration Rate

In [0]:
all_psb_comb = 1
for pn in tuning_args:
  all_psb_comb *= len(param_range[pn])
er = len(accuracy_lookuptb)/all_psb_comb

In [62]:
er

0.013111888111888112

# Find The Best Configuration

In [60]:
best_idx = np.argmax(testing_his_max_reward.values())
best_cfg = list(testing_his_max_reward.keys())[best_idx]
best_acc = list(testing_his_max_reward.values())[best_idx]
best_cfg, best_acc

("{'C': 7850, 'gamma': 101.0}", 0.892679548890327)

## Grid Search All Possible Accuracy

In [73]:
actual_acc_dict = {}
for c in tqdm(param_range["C"]):
  for g in tqdm(param_range["gamma"]):
    tmp_args = a.args.copy()
    tmp_args["C"] = c
    tmp_args["gamma"] = g
    actual_acc_dict[str(tmp_args)] = calReward(tmp_args, mdl, XY)

## 

## Performance Evaluation

In [75]:
acc_better_than = sum(np.array(list(actual_acc_dict.values())) > best_acc)/len(actual_acc_dict)
actual_best_conf = list(actual_acc_dict.keys())[np.argmax(list(actual_acc_dict.values()))]
actual_best_acc = np.max(list(actual_acc_dict.values()))

print("The configuration found by RL is top %f of the overall accuracy by spending %f of the time needed for grid search"%(acc_better_than, er))
print("The configuration found by RL is %s, the actual best configuration is %s"%(str(best_cfg), str(actual_best_conf)))
print("The accuracy found by RL is %f, the actual best accuracy is %f"%(best_acc, actual_best_acc))

The configuration found by RL is top 0.235334 of the overall accuracy by spending 0.013112 of the time needed for grid search
The configuration found by RL is {'C': 7850, 'gamma': 101.0}, the actual best configuration is {'C': 9900, 'gamma': 55.0}
The accuracy found by RL is 0.892680, the actual best accuracy is 0.898705
